#### First, we import some libraries

In [30]:
# for arrays
import numpy as np

# for dataframes
import pandas as pd

# plots
import matplotlib.pyplot as plt
# high-level plots
import seaborn as sns

# statistics
import scipy.stats as sc
# hierarchical clustering, clusters
from scipy.cluster.hierarchy import linkage, cut_tree, leaves_list
from scipy import stats
# statistical tests
from scipy.stats import mannwhitneyu

# machine learning library
# Principal Component Analysis - determine new axis for representing data
from sklearn.decomposition import PCA
# Random Forests -> vote between decision trees
# Gradient boosting -> instead of a vote, upgrade the same tree
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, GradientBoostingClassifier
# Decision Tree
from sklearn.tree import DecisionTreeRegressor, plot_tree
# To add interactions in linear regressions models
from sklearn.preprocessing import PolynomialFeatures
# Elasticnet is an hybrid method between ridge and Lasso
from sklearn.linear_model import LinearRegression, ElasticNet
# To separate the data into training and test
from sklearn.model_selection import train_test_split
# Simple clustering (iterative steps)
from sklearn.cluster import KMeans
# get interactions of features
from sklearn.preprocessing import PolynomialFeatures
# loss functions and metrics
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# we use it to interact with the file system
import os
# compute time
from time import time

# statistics
import scipy.stats as sc
# hierarchical clustering, clusters
from scipy.cluster.hierarchy import linkage, cut_tree, leaves_list
from scipy import stats
# statistical tests
from scipy.stats import mannwhitneyu

# no warning
import warnings
warnings.filterwarnings("ignore")

In [2]:
data_dir = "../data/"

# the name of the systems we are testing
name_systems = ["nodejs", "poppler", "xz", "x264", "gcc", "lingeling", "sqlite", "imagemagick"]

# final results
data = dict()

inputs_name = dict()
inputs_count = dict()
inputs_perf = dict()
inputs_feat = dict()
inputs_categ = dict()
inputs_prop = dict()
inputs_num = dict()


# name of the performance properties
inputs_perf["gcc"] = ["size", "ctime", "exec"]
inputs_perf["imagemagick"] = ["size", "time"]
inputs_perf["lingeling"] = ["conflicts", "cps", "reductions"]
inputs_perf["nodejs"] = ["ops"]
inputs_perf["poppler"] = ["size", "time"]
inputs_perf["sqlite"] = ["q"+str(i+1) for i in range(15)]
inputs_perf["x264"] = ["size", "kbs", "fps", "etime", "cpu"]
inputs_perf["xz"] = ["size", "time"]

# name of features for each system
inputs_feat["gcc"] = ["optim","-floop-interchange","-fprefetch-loop-arrays","-ffloat-store","-fno-asm"]
inputs_feat["imagemagick"] = ["memory_r", "posterize_r", "gaussian-blur", "thread", "quality"]
inputs_feat["lingeling"] = ["--boost", "--carduse", "--decompose", "--gluescale", "--lkhd", "--memlim", 
"--minimize", "--prbsimple", "--sweepirr", "--sweepred"]
inputs_feat["nodejs"] = ["--jitless", "--experimental-wasm-modules", "--experimental-vm-modules",
                         "--preserve-symlinks-main","--no-warnings","--node-memory-debug"]
inputs_feat["poppler"] = ["format","j","jp2","jbig2","ccitt"]
inputs_feat["sqlite"] = ["-deserialize", "-memtrace", "-maxsize", "-append", "-output"]
inputs_feat["x264"] = ["cabac", "ref", "deblock", "analyse", "me", "subme", "mixed_ref", "me_range", "trellis", 
                "8x8dct", "fast_pskip", "chroma_qp_offset", "bframes", "b_pyramid", "b_adapt", "direct", 
                "weightb", "open_gop", "weightp", "scenecut", "rc_lookahead", "mbtree", "qpmax", "aq-mode"]
inputs_feat["xz"] = ["memory","format","level","depth"]

# just to isolate the options that have categorial values
# because it is more difficult to handle for ML algorithms
inputs_categ["gcc"] = ["optim"]
inputs_categ["imagemagick"] = []
inputs_categ["lingeling"] = []
inputs_categ["nodejs"] = []
inputs_categ["poppler"] = ["format"]
inputs_categ["sqlite"] = []
inputs_categ["x264"] = ['analyse', 'me', 'direct', 'deblock', 'b_adapt', 'b_pyramid', 
                        'open_gop', 'rc_lookahead', 'scenecut', 'weightb']
inputs_categ["xz"] = ['memory', 'format']

for ns in name_systems:
    
    data_path = data_dir+ns+'/'
    
    inputs_prop[ns] = pd.read_csv(data_path+'/others/properties.csv').set_index('id')
    
    inputs = [str(name)+'.csv' for name in inputs_prop[ns]['name']]

    inputs_name[ns] = inputs
    inputs_count[ns] = len(inputs)
    
    inputs_num[ns] = np.setdiff1d(inputs_feat[ns], inputs_categ[ns])
    
    for i in range(len(inputs)):
        loc = data_path+inputs[i]
        df = pd.read_csv(loc)
        if inputs_categ[ns] != []:
            data[ns, i] = df[inputs_num[ns]].join(pd.get_dummies(df[inputs_categ[ns]])).join(df[inputs_perf[ns]])
        else:
            data[ns, i] = df[inputs_num[ns]].join(df[inputs_perf[ns]])

#### Load Model Shift Results

In [31]:
# defined above :  
# name_systems = ["nodejs", "poppler", "xz", "x264", "gcc", "lingeling", "sqlite", "imagemagick"]

prop_train_config = [0.1* k for k in range(1,10)]
proportion_inputs = [0.1* k for k in range(1,10)]
methods = ['random', 'same_input_grp', 'closest_properties', 'max_perf_corr']
repetitions = 10

res = pd.DataFrame(columns = ['system', 'perf', 'proportion_train_config', 'source_selection_method', 
                              'prop_inputs_source', 'id repetition', 'mape', 'training_time'])

for ns in name_systems:
    for perf in inputs_perf[ns]:
        df = pd.read_csv('../results/'+ns+'/MS_'+perf+'2.csv', index_col=0)
        df['system'] = [ns for k in range(df.shape[0])]
        df['perf'] = [perf for k in range(df.shape[0])]
        res = pd.concat([res, df], axis=0)

In [32]:
res.columns

Index(['system', 'perf', 'proportion_train_config', 'source_selection_method',
       'prop_inputs_source', 'id repetition', 'mape', 'training_time'],
      dtype='object')

In [34]:
res.groupby(['source_selection_method']).median()[['mape', 'training_time']]

,mape,training_time
source_selection_method,,
closest_properties,4.173739,0.052628
max_perf_corr,3.823381,0.067205
random,5.219429,0.015127
same_input_grp,4.700469,0.015454


In [35]:
res.groupby(['proportion_train_config']).median()[['mape', 'training_time']]

,mape,training_time
proportion_train_config,,
0.1,7.729175,0.028512
0.2,5.114500,0.029250
0.3,4.471608,0.029904
0.4,4.136383,0.028764
0.5,3.837135,0.030094
0.6,3.867606,0.031234
0.7,3.673543,0.030791
0.8,3.413774,0.031437
0.9,3.072813,0.030269


In [36]:
res.groupby(['prop_inputs_source']).median()[['mape', 'training_time']]

,mape,training_time
prop_inputs_source,,
0.1,4.389779,0.020595
0.2,4.299278,0.025836
0.3,4.482063,0.029917
0.4,4.437995,0.033658
0.5,4.422039,0.037604
0.6,4.401878,0.041229
0.7,4.397112,0.043997
0.8,4.409274,0.047500
0.9,4.523836,0.045094


In [37]:
res.groupby(['system']).median()[['mape', 'training_time']]

,mape,training_time
system,,
gcc,9.899953,0.018040
imagemagick,3.221763,0.066516
lingeling,4.237170,0.039670
nodejs,4.418772,0.078853
poppler,5.189736,0.060486
sqlite,2.343105,0.022153
x264,11.129514,0.100752
xz,1.214103,0.017973


In [38]:
res.groupby(['system', 'perf']).median()[['mape', 'training_time']]

mape  training_time
system      perf                                
gcc         ctime       12.316061       0.016159
            exec        15.925827       0.019839
            size         1.250165       0.017646
imagemagick size         0.541729       0.074789
            time         5.061139       0.061272
lingeling   conflicts    4.929528       0.041268
            cps          4.991192       0.042046
            reductions   3.191439       0.035015
nodejs      ops          4.418772       0.078853
poppler     size         0.717972       0.059994
            time         6.745772       0.060764
sqlite      q1           1.029258       0.020404
            q10          2.182363       0.025967
            q11          2.806696       0.025684
            q12          2.476261       0.019284
            q13          3.353348       0.023835
            q14          2.408763       0.024042
            q15          2.299459       0.021233
            q2           5.149361       0.023287
            q3           2.750891       0.021841
            q4           2.552631       0.022440
            q5           1.102111       0.018337
            q6           3.499479       0.018281
            q7           0.816326       0.022528
            q8           2.918459       0.021431
            q9           1.310584       0.023145
x264        cpu         10.826974       0.099111
            etime       12.686922       0.097943
            fps         14.674584       0.108488
            kbs          8.550160       0.100142
            size         8.349566       0.102985
xz          size         0.000896       0.019168
            time         4.999220       0.016973

In [39]:
res

,system,perf,proportion_train_config,source_selection_method,prop_inputs_source,id repetition,mape,training_time
0,nodejs,ops,0.1,random,0.1,0,2.147911,0.027209
1,nodejs,ops,0.1,random,0.1,1,9.538719,0.014889
2,nodejs,ops,0.1,random,0.1,2,37.098987,0.014795
3,nodejs,ops,0.1,random,0.1,3,5.815130,0.013550
4,nodejs,ops,0.1,random,0.1,4,107.468880,0.014748
...,...,...,...,...,...,...,...,...
3235,imagemagick,time,0.9,max_perf_corr,0.9,5,4.660732,0.431938
3236,imagemagick,time,0.9,max_perf_corr,0.9,6,4.660732,0.431868
3237,imagemagick,time,0.9,max_perf_corr,0.9,7,0.966516,0.427146
3238,imagemagick,time,0.9,max_perf_corr,0.9,8,2.161955,0.421922


In [40]:
rand_vals = res.query("source_selection_method == 'random'")["mape"]
perf_vals =res.query("source_selection_method == 'max_perf_corr'")["mape"]
input_vals =res.query("source_selection_method == 'closest_properties'")["mape"]
grp_vals =res.query("source_selection_method == 'same_input_grp'")["mape"]

stats.wilcoxon(rand_vals, perf_vals)

WilcoxonResult(statistic=128363725.0, pvalue=0.0)

In [41]:
stats.wilcoxon(rand_vals, input_vals)

WilcoxonResult(statistic=140769985.5, pvalue=9.593202735705998e-183)

In [42]:
stats.wilcoxon(grp_vals, rand_vals)

WilcoxonResult(statistic=163303554.5, pvalue=1.0929775409001952e-26)

#### Load Model Shift Results

In [39]:
# defined above :  
# name_systems = ["nodejs", "poppler", "xz", "x264", "gcc", "lingeling", "sqlite", "imagemagick"]

prop_train_config = [0.1* k for k in range(1,10)]
proportion_inputs = [0.1* k for k in range(1,10)]
repetitions = 10

res = pd.DataFrame(columns = ['system', 'perf', 'proportion_train_config',
                              'prop_inputs_source', 'id repetition', 'mape', 'training_time'])

for ns in name_systems:
    for perf in inputs_perf[ns]:
        df = pd.read_csv('../results/'+ns+'/SimpleMulti_'+perf+'.csv', index_col=0)
        df['system'] = [ns for k in range(df.shape[0])]
        df['perf'] = [perf for k in range(df.shape[0])]
        res = pd.concat([res, df], axis=0)

In [40]:
res.groupby(['proportion_train_config']).median()[['mape', 'training_time']]

,mape,training_time
proportion_train_config,,
0.1,26.901523,0.120514
0.2,29.121873,0.125653
0.3,28.600386,0.132062
0.4,29.295293,0.138169
0.5,29.775873,0.144161
0.6,28.913140,0.151158
0.7,28.944781,0.158405
0.8,28.432445,0.167204
0.9,19.183516,0.177483


In [41]:
res.groupby(['prop_inputs_source']).median()[['mape', 'training_time']]

,mape,training_time
prop_inputs_source,,
0.1,28.153647,0.105073
0.2,27.005714,0.114082
0.3,26.811525,0.124329
0.4,27.613291,0.133847
0.5,27.853977,0.144099
0.6,26.999080,0.152682
0.7,27.454269,0.162966
0.8,27.503441,0.173721
0.9,28.423853,0.183285
